In [1]:
## Standard Stuff
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import dask.array as da
import numcodecs

## HEALPix Specific
import healpix as hp
import easygems.healpix as egh
import easygems.remap as egr

import intake     # For catalogs
import zarr

# Ilan
from icecream import ic
import nc_time_axis

def worldmap(var, title='', cbar_title='', **kwargs):
    #projection = ccrs.Robinson(central_longitude=-135.5808361)
    projection = ccrs.Robinson(central_longitude=0)
    fig, ax = plt.subplots(
        figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_global()
    ax.set_title(title)

    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.8, label=cbar_title)
    ax.add_feature(cf.COASTLINE, linewidth=0.8)
    ax.add_feature(cf.BORDERS, linewidth=0.4)
    
def usmap(var, title='', cbar_title='', **kwargs):
    #projection = ccrs.Robinson(central_longitude=-135.5808361)
    #projection = ccrs.Robinson(central_longitude=-90)
    projection = ccrs.PlateCarree()
    fig, ax = plt.subplots(
        figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_extent([-110, -60, 20, 45])
    
    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.8, label=cbar_title)
    ax.set_title(title)
    ax.add_feature(cf.COASTLINE, linewidth=0.8)
    #ax.coastlines(linewidth=0.8)
    ax.add_feature(cf.BORDERS, linewidth=0.4)
    ax.add_feature(cf.STATES, linewidth=0.4)

In [2]:
# set up dask
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
pbar = ProgressBar()
pbar.register()
# cluster = LocalCluster()
# client = Client(cluster)
# client

In [3]:
catfn='/home/tmerlis/hackathon/hackathon_cat_may14_main.yaml'

combo_cat = intake.open_catalog(catfn)
# ICON and IFS
print (list(combo_cat)) 

['xsh24_coarse', 'xsh24_native', 'xsh21_coarse', 'scream2D_hrly', 'scream_ne120', 'scream_lnd', 'ifs_fesom', 'icon_3hp003']


In [4]:
def preprocess(ds):
    res = ds \
        .assign(wind=lambda x: np.sqrt(x['ua']**2 + x['va']**2)) \
        .assign(height=lambda x: x['zg']) # geopotential height (basically height)
    return res
    
# select zoom level and the part of the combined catalog you're interested in
# coarse stores are available at zoom 7 ~50km and lower
#zoom_select = 7 # Wind speeds are messed up for zoom 7 and less
ifs = combo_cat.ifs_fesom().to_dask().pipe(egh.attach_coords).pipe(preprocess)
icon = combo_cat.icon_3hp003().to_dask().pipe(egh.attach_coords).pipe(preprocess)
print(ifs)
print(icon)

/home/iv4111/.conda/envs/easy25/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),
/home/iv4111/.conda/envs/easy25/lib/python3.12/site-packages/intake_xarray/base.py:21: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  'dims': dict(self._ds.dims),


<xarray.Dataset> Size: 4TB
Dimensions:   (time: 10201, cell: 196608, level_snow: 5, level: 25)
Coordinates:
    lat       (cell) float64 2MB 0.2984 0.5968 0.5968 ... -0.5968 -0.2984
  * level     (level) float32 100B 1.0 5.0 10.0 20.0 ... 925.0 950.0 975.0 1e+03
    lon       (cell) float64 2MB 45.0 45.35 44.65 45.0 ... 315.4 314.6 315.0
  * time      (time) datetime64[ns] 82kB 2020-01-01 ... 2021-03-01
    crs       int64 8B 0
  * cell      (cell) int64 2MB 0 1 2 3 4 ... 196603 196604 196605 196606 196607
Dimensions without coordinates: level_snow
Data variables: (12/83)
    100si     (time, cell) float32 8GB dask.array<chunksize=(168, 4096), meta=np.ndarray>
    100u      (time, cell) float32 8GB dask.array<chunksize=(168, 4096), meta=np.ndarray>
    100v      (time, cell) float32 8GB dask.array<chunksize=(168, 4096), meta=np.ndarray>
    10si      (time, cell) float32 8GB dask.array<chunksize=(168, 4096), meta=np.ndarray>
    2d        (time, cell) float32 8GB dask.array<chunksize=(

In [8]:
def get_lljs(c_winds, c_heights):
    """
    Input: one cell of data, dim is [plev]
    Output: dataset with 3 fields
        1. mask [time, cell] -> True for jet, False for no
        2. height [time, cell] -> height of the LLJ [m] (will always have data)
        2. strength [time, cell] -> strength of the jet core [m/s] (will always have data)
    """
    # Need increasing heights for np.interp to work
    good_heights = c_heights >= 0
    c_heights, c_winds = c_heights[good_heights], c_winds[good_heights]
    sort_idx = np.argsort(c_heights)
    c_heights, c_winds = c_heights[sort_idx], c_winds[sort_idx]
    # Step 1 -> interpolate winds to specific height levels
    heights = np.arange(10, 3010, 10) # 100m threshold
    winds = np.interp(heights, c_heights, c_winds)
    core_idx = np.nanargmax(winds)
    # Get core properties
    core_height = heights[core_idx]
    if core_height > 1000: # will lead to index error
        return (0, np.nan, np.nan)
    core_speed = winds[core_idx]
    # Get layer properties
    buffer_idx = core_idx + 50 # 500 m buffer @ 10 m spacing
    buffer_speed = winds[buffer_idx]
    shear = np.gradient(winds, 10)[buffer_idx]
    # Checks
    jet = ((core_speed - buffer_speed) > 2) \
        & (core_height <= 1000) \
        & (core_height >= 50) \
        & (core_speed >= 10) \
        & (shear < 0.005)
    if jet:
        return (1, core_height, core_speed)
    else:
        return (0, np.nan, np.nan)

def apply_lljs(wind, height, p_name):
    jet_mask, jet_height, jet_speed = xr.apply_ufunc(
        get_lljs,
        wind.chunk({p_name:-1}),
        height.chunk({p_name:-1}),
        input_core_dims=[[p_name], [p_name]],
        output_core_dims=[[], [], []],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[bool, float, float],
    )
    return xr.merge([
        jet_mask.rename('mask'),
        jet_height.rename('height'), 
        jet_speed.rename('speed')
    ])

In [ ]:
for ds, name, p_label in zip([ifs, icon], ['IFS', 'ICON'], ['level', 'pressure']):
    us = ((ds.lat <= 50) & (ds.lat >= 20) & (ds.lon >= -130+360) & (ds.lon <= -60+360))
    ds_us = ds.isel(cell=us)
    llj = apply_lljs(ds_us.wind, ds_us.height, p_label)
    llj['crs'].attrs = ds_us['crs'].attrs.copy() # need to copy attributes so healpix doesn't mess up
    llj.to_netcdf(f'/scratch/cimes/iv4111/hk25-data/llj_{name}.h5')

[################                        ] | 40% Completed | 14m 40ss


In [ ]:
for ds, name in zip([ifs, icon], ['IFS', 'ICON']):
    llj = xr.open_dataset(f'/scratch/cimes/iv4111/hk25-data/llj_{name}.h5', chunks='auto')
    freq = llj.mask.sum('time')/len(llj.time)*100
    ### Only select regions where they occur >5% of the time ###
    freq = freq.where(freq>5, np.nan)
    llj = llj.where(freq>5, np.nan)
    title = f'{name}'
    usmap(freq, title=f'LLJ Occurrence {title}', cbar_title='Frequency [%]', vmin=0)
    # Speed
    usmap(llj.speed.mean('time'), title=f'LLJ Speed {title}', cbar_title=r'Mean Speed of LLJ [m s$^{-1}$]', vmin=0, cmap='plasma')
    # Jet height
    usmap(llj.height.mean('time'), title=f'LLJ Height {title}', cbar_title=r'Mean Height of LLJ [m]', vmin=0, cmap='cividis')